In [ ]:
import pandas as pd
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import LSTM, GlobalAveragePooling1D, Dense
from keras.models import Sequential
from keras.activations import sigmoid, relu
from sklearn.model_selection import train_test_split

datapath = "./Input/"
types = ["BodyWeightSquats", "PushUps"]
extension = "*.csv"

In [ ]:
main_df = pd.DataFrame()

for count, cur in enumerate(types):
    current_glob = glob.glob(datapath+cur+"/"+extension)
    df = pd.DataFrame()
    li = []

    for filename in current_glob:
        df = pd.read_csv(filename, index_col=None, header=0)
        p = re.compile("\\\\v_(.*)\.csv")
        filename = p.findall(filename)[0]
        df.insert(0, 'example', filename)
        df.insert(0, 'exercise', cur)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    if count == 0:
        main_df = pd.DataFrame(columns=frame.columns, data=frame)
    else:
        main_df = pd.concat([main_df, frame])


print(main_df.head(100000))

In [ ]:
max_frames = main_df.frame.max() + 1
print(max_frames)
plt.boxplot(main_df.frame)

In [ ]:
print(len(pd.unique(main_df.example)))
#print(main_df.head())
yhat = main_df.groupby(by="example").exercise.max()

yhat = pd.get_dummies(yhat)
print(yhat)

In [ ]:
video_frames = []

for video in main_df.groupby(by="example"):
    current = video[1]
    current = current.drop(columns=["exercise","frame","example"])
    current = current.to_numpy()
    #print(current.shape)
    #print(type(current))
    padded = np.ones((max_frames, 132))
    padded *= -2
    padded[0:current.shape[0],:] = current
    video_frames.append(padded)

x = np.stack(video_frames, axis=0)
print(x.shape)
#214 is the video count
#267 is max length of video
#132 is the number of features per frame

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, yhat)

print(x_train.shape)
print(x_test.shape)

model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation="softmax"))
model.compile(optimizer="adam", loss="CategoricalCrossentropy")

model.fit(x, yhat)
model.summary()